# Imports

## Import models

### Common imports

In [3]:
from PIL import Image
import requests
import torch

### CLIP

In [4]:
from transformers import CLIPProcessor, CLIPModel

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### EasyOCR

In [5]:
import easyocr
from PIL import ImageDraw

## General imports

In [6]:
import numpy as np
import statistics

# Global declarations

In [7]:
allergens = {
    "gluten": 0,
    "eggs": 1,
    "milk": 2,
    "nuts": 3,
    "peanuts": 4,
    "soja": 5,
    "molluscs": 6,
    "fish": 7,
    "lupin": 8,
    "crustaceans": 9,
    "sesame": 10,
    "mustard": 11,
    "celery": 12,
    "sulphites": 13
}

nb_allergens = len(allergens)

# Body

## CLIP prediction

In [8]:
model_CLIP = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor_CLIP = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
def predict_CLIP(image, allergen):
    image = Image.open(image)
    match allergen:
        case 0:
            inputs = processor_CLIP(text=["ingredient list of a product containing wheat or gluten", "ingredient list of a product without wheat nor gluten"], images=image, return_tensors="pt", padding=True)
        case 1:
            inputs = processor_CLIP(text=["ingredient list of a product containing eggs", "ingredient list of a product without eggs"], images=image, return_tensors="pt", padding=True)
        case 2:
            inputs = processor_CLIP(text=["ingredient list of a product containing milk", "ingredient list of a product without milk"], images=image, return_tensors="pt", padding=True)
        case 3:
            inputs = processor_CLIP(text=["ingredient list of a product containing nuts", "ingredient list of a product without nuts"], images=image, return_tensors="pt", padding=True)
        case 4:
            inputs = processor_CLIP(text=["ingredient list of a product containing peanuts", "ingredient list of a product without peanuts"], images=image, return_tensors="pt", padding=True)
        case 5:
            inputs = processor_CLIP(text=["ingredient list of a product containing soya", "ingredient list of a product without soya"], images=image, return_tensors="pt", padding=True)
        case 6:
            inputs = processor_CLIP(text=["ingredient list of a product containing molluscs", "ingredient list of a product without molluscs"], images=image, return_tensors="pt", padding=True)
        case 7:
            inputs = processor_CLIP(text=["ingredient list of a product containing fish", "ingredient list of a product without fish"], images=image, return_tensors="pt", padding=True)
        case 8:
            inputs = processor_CLIP(text=["ingredient list of a product containing lupin", "ingredient list of a product without lupin"], images=image, return_tensors="pt", padding=True)
        case 9:
            inputs = processor_CLIP(text=["ingredient list of a product containing crustaceans", "ingredient list of a product without crustaceans"], images=image, return_tensors="pt", padding=True)
        case 10:
            inputs = processor_CLIP(text=["ingredient list of a product containing sesame", "ingredient list of a product without sesame"], images=image, return_tensors="pt", padding=True)
        case 11:
            inputs = processor_CLIP(text=["ingredient list of a product containing mustard", "ingredient list of a product without mustard"], images=image, return_tensors="pt", padding=True)
        case 12:
            inputs = processor_CLIP(text=["ingredient list of a product containing celery", "ingredient list of a product without celery"], images=image, return_tensors="pt", padding=True)
        case 13:
            inputs = processor_CLIP(text=["ingredient list of a product containing sulphites", "ingredient list of a product without sulphites"], images=image, return_tensors="pt", padding=True)

    outputs = model_CLIP(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

    return round(probs[0][0].item())

In [10]:
def predict_allergens_CLIP(image):
    prediction = []
    for allergen in range(0, 14):
        prediction.append(predict_CLIP(image, allergen))

## EasyOCR Prediction

In [11]:
reader = easyocr.Reader(['en'], gpu=False)  # English

Using CPU. Note: This module is much faster with a GPU.


In [12]:
def predict_allergens_EasyOCR(image):
    # ------------ Extract text ---------------------
    image_path = 'CLIP/feature_engineering_dataset/enhanced_data/002.jpg'
    result = reader.readtext(image_path)
    text = ' '.join(item[1] for item in result)
    text = text.lower()
    # -----------------------------------------------

    prediction = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    print(text)

    if "gluten" in text or "wheat" in text:
        prediction[allergens['gluten']] = 1
    if "egg" in text:
        prediction[allergens['eggs']] = 1
    if "milk" in text or "whey" in text:
        prediction[allergens['milk']] = 1
    if "nut" in text:
        prediction[allergens['nuts']] = 1
    if "peanut" in text:
        prediction[allergens['peanuts']] = 1
    if "soja" in text or "soy" in text:
        prediction[allergens['soja']] = 1
    if "mollusc" in text:
        prediction[allergens['molluscs']] = 1
    if "fish" in text or "tuna" in text or "salmon" in text or "cod" in text or "tilapia" in text or "sardine" in text:
        prediction[allergens['fish']] = 1
    if "lupin" in text:
        prediction[allergens['lupin']] = 1
    if "crustacean" in text:
        prediction[allergens['crustaceans']] = 1
    if "sesame" in text:
        prediction[allergens['sesame']] = 1
    if "mustard" in text:
        prediction[allergens['mustard']] = 1
    if "celery" in text:
        prediction[allergens['celery']] = 1
    if "sulphite" in text:
        prediction[allergens['sulphites']] = 1
        print("hey")

    return prediction

## Read Excel File

In [14]:
import pandas as pd

C:\Users\nizon\AppData\Local\Temp\ipykernel_5352\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [38]:
excel_file = pd.read_excel('labellisation.xlsx')
excel_file = excel_file.iloc[0:219, 1:15]

excel_file

,gluten,egg,milk,nuts,peanuts,soya,molluscs,fish,lupin,crustaceans,sesame,mustard,celery,sulphites
0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
216,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
217,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Raw Data Performance Comparison

In [40]:
results_CLIP = pd.DataFrame()

for image in range(1, 21):
    prediction_CLIP = predict_allergens_CLIP(f"datasets/raw_dataset/{image:03d}.jpg")
    
results_CLIP = results_CLIP.append(pd.Series(prediction_CLIP, name=f"{image}"))

results_CLIP

AttributeError: 'DataFrame' object has no attribute 'append'